# Authed flows with Browser Use and Hyperbrowser

In this cookbook, we'll demonstrate how to use User Profiles with Hyperbrowser's browser use agent. In this example, we'll be using Instagram as an example. This combination allows you to:

- Navigate to Instagram profiles without manual intervention
- Interact with content (like posts, follow accounts, etc.)
- Perform these actions from a persistent browser profile that maintains your login session

Unlike traditional web automation that requires complex selectors and frequent maintenance, this approach uses AI to understand and navigate Instagram's interface visually - just like a human would. This makes the automation more resilient to UI changes and significantly easier to implement.

## Prerequisites

Before starting, you'll need:

1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai) if you don't have one)
2. An Instagram account already logged in through a Hyperbrowser profile
3. Python 3.9+ installed

Your API key should be stored in a `.env` file in the same directory as this notebook:

```
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
```

**Important Note**: This notebook demonstrates browser automation techniques for educational purposes. When using automation with social platforms like Instagram, always respect their terms of service and community guidelines.

## Step 1: Import Libraries and Initialize Hyperbrowser

We start by importing the necessary packages and initializing our Hyperbrowser client. The key components we need are:

- `AsyncHyperbrowser`: The asynchronous API client for Hyperbrowser
- `StartBrowserUseTaskParams`: Parameters for configuring our browser automation task
- `CreateSessionParams` and `CreateSessionProfile`: For setting up persistent browser sessions with saved login state

In [ ]:
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models import (
    StartBrowserUseTaskParams,
    CreateSessionParams,
    CreateSessionProfile,
)

from IPython.display import Markdown, display
from pydantic import BaseModel

load_dotenv()

## Step 2: Create a Hyperbrowser Client

Next, we initialize the Hyperbrowser client with our API key loaded from the environment variables. This client will handle all communication with the Hyperbrowser API.

In [10]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))

## Step 3: Set Up Browser Profile

For Instagram automation, we need a persistent browser profile that maintains login state. This section contains code for:

1. Using an existing profile ID where you're already logged into Instagram
2. Creating a new profile and launching a live session where you can manually log in

When working with social media automation, persistent profiles are essential because they:
- Store cookies and authentication data between sessions
- Maintain preferences and settings
- Reduce the risk of triggering security mechanisms that detect new browser environments

**First-time setup**: If you're running this notebook for the first time, uncomment the profile creation code, run it, and manually log into Instagram using the live URL that gets printed. After logging in, save the profile ID for future use.

In [ ]:
profile_id = "[If a profile already exists, use it here]"
### If you are using this notebook for the first time, uncomment the following lines and run them
# from hyperbrowser.models import CreateSessionParams, CreateSessionProfile

# # profile = await hb.profiles.create()
# # print(profile)
# # profile_id = profile.id
# session = await hb.sessions.create(
#     CreateSessionParams(
#         use_proxy=True,
#         profile=CreateSessionProfile(id=profile_id, persist_changes=True),
#     )
# )
# print(session.live_url)

## Step 4: Clean Up Sessions (Optional)

This cell can be used to stop active browser sessions when they're no longer needed. Running unnecessary sessions can consume resources, so it's good practice to clean up after you're done with manual login or debugging.

In [12]:
# await hb.sessions.stop(session.id)

## Step 5: Define the Instagram Interaction Function

Now we'll create a function that performs a specific Instagram interaction - liking the most recent post from a given user handle. This function demonstrates the power of Hyperbrowser's browser_use agent by:

1. Using natural language to describe the task ("Go to the instagram handle..., scroll down, and like their latest post")
2. Enabling computer vision capabilities so the agent can "see" and interact with visual elements
3. Using our persistent profile to maintain Instagram login state

The function is remarkably simple compared to traditional web automation code, which would require complex selectors, waiting logic, and error handling.

In [13]:
async def like_first_post_on_instagram(user_handle:str):
    resp = await hb.beta.agents.browser_use.start_and_wait(
        StartBrowserUseTaskParams(
            task=f"Go to the instagram handle {user_handle}, scroll down, and like their latest post.",
            use_vision=True,
            session_options=CreateSessionParams(
                profile=CreateSessionProfile(id=profile_id)
            ),
        )
    )
    if resp.data is not None:
        return resp.data.final_result
    return None

## Step 6: Execute the Automation and Display Results

Finally, we'll run our function with the National Geographic (@natgeo) Instagram handle as an example. The agent will:

1. Navigate to the National Geographic Instagram profile
2. Scroll down to find the latest post
3. Like the post by clicking the like button
4. Return a confirmation message

Behind the scenes, the agent is making real-time decisions about how to navigate Instagram's interface, locate buttons, and perform interactions - just like a human would.

In [14]:
response = await like_first_post_on_instagram("@natgeo")
if response is not None:
    display(Markdown(response))
else:
    print("No response from the agent")

Successfully navigated to the instagram handle @natgeo, scrolled down, and liked their latest post.

## Conclusion

This cookbook demonstrates a new approach for browser automations - using AI agents that understand visual interfaces and natural language instructions rather than brittle, selector-based automation. By leveraging persistent user profiles, this approach also solves the problem of any form of authentication. This kind of method has a few key advantages, 

1. **Resilience**: The AI can adapts to UI changes since it operates visually, and based on structure rather than relying on specific HTML structures
2. **State Management**: Persistent profiles maintain login sessions and user preferences across automation runs
3. **Natural Behavior**: The agent navigates and interacts with websites in a human-like manner

While this example focuses on a simple interaction, the same approach can be extended to more complex workflows requiring authenticated sessions and consistent user states.

All this said, Hyperbrowser encourages responsible automation that respects websites' Terms of Service and rate limits.

## Relevant Links
- [Hyperbrowser](https://hyperbrowser.ai)
- [Hyperbrowser Documentation](https://docs.hyperbrowser.ai)